# Install Dependencies

In [23]:
!pip install torch torchvision torchaudio

  Using cached torch-2.0.1-cp310-cp310-win_amd64.whl (172.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0
    Uninstalling torch-2.1.0:
      Successfully uninstalled torch-2.1.0


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Admin\\anaconda3\\Lib\\site-packages\\~orch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



In [24]:
!pip install stable-baselines3[extra] protobuf==3.20.*

In [25]:
!pip install mss pydirectinput pytesseract

In [26]:
pip install gym

Note: you may need to restart the kernel to use updated packages.


In [27]:
!pip install stable-baselines3[extra]

In [28]:
!pip install matplotlib

In [32]:
from mss import mss  # for screen capture
import pydirectinput  # sending commands
import cv2  # frame processing
import numpy as np  # transformational framework
import pytesseract  # OCR for GAME OVER extraction
from matplotlib import pyplot as plt  # for visualization
import time  # for pauses
from gym import Env  # Environment componenets
from gym.spaces import Box, Discrete

# Build the Environment

In [33]:
class WebGame(Env):
    
    # setup environment, action & observation
    def __init__(self):
        super().__init__()
        # Setup spaces
        self.observation_space = Box(low=0, high=255, shape=(1,83,100), dtype=np.uint8)
        self.action_space = Discrete(3)
        # Capture game frames
        self.cap = mss()
        self.game_location = {'top': 300, 'left': 0, 'width': 600, 'height': 500}
        self.done_location = {'top': 280, 'left': 410, 'width': 660, 'height': 70}
    
    # to do something
    def step(self, action):
        # action keys 1 -> Jump, 2-> Duck, 3 -> No operation
        actions_map = {
            0: 'space',
            1: 'down',
            2: 'no_op'
        }
        if action != 2:
            pydirectinput.press(actions_map[action])
        # Check if game is done
        done, done_screen = env.get_done()
        # Get new observation
        new_observation = self.get_observation()
        # Reward - get a point for every frame
        reward = 1
        # Info dictionary
        info = {}
        
        return new_observation, reward, done, info
    
    # visualize the game
    def render(self):
        cv2.imshow('Game', np.array(self.cap.grab(self.game_location))[:, :, :3])
        if cv2.waitKey(1) & 0xFF == ord('q'):
            self.close()
    
    # restart
    def reset(self):
        time.sleep(1)
        pydirectinput.click(x=150, y=150)
        pydirectinput.press('space')
        return self.get_observation()
    
    # get the part of the observation of the game
    def get_observation(self):
        # get screen capture of game
        raw = np.array(self.cap.grab(self.game_location))[:, :, :3]  # height, width, rgb channels
        # Grayscale
        gray = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY)
        # Resize
        resized = cv2.resize(gray, (100, 83))
        # Add channels
        channels = np.reshape(resized, (1, 83, 100))
        return channels
    
    # check for game over
    def get_done(self):
        done_cap = np.array(self.cap.grab(self.done_location))
        done_strings = ['GAME', 'GAHE','GANE']
        done=False
        # if np.sum(done_cap) < 44300000:
        #     done = True
        done = False
        res = pytesseract.image_to_string(done_cap)[:4]
        if res in done_strings:
            done = True
        return done, done_cap
    
    #close 
    def close(self):
        cv2.destroyAllWindows()
    
    

In [34]:
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"

In [31]:
env = WebGame()

In [ ]:
obs = env.get_observation()

In [ ]:
plt.imshow(cv2.cvtColor(obs[0], cv2.COLOR_BGR2RGB)) 

In [ ]:
done, done_screen = env.get_done()
plt.imshow(np.array(done_screen))
print(done)

In [ ]:
pytesseract.image_to_string(done_screen)[:4]

In [ ]:
env.reset()

In [ ]:
env.render()

In [ ]:
# Test for 10 games
for episode in range(10):
    obs = env.reset()
    done = False
    total_rewards = 0
    while not done:
        obs, reward, done, info = env.step(env.action_space.sample())
        total_rewards += reward
    print(f'Total reward for episode {episode} is {total_rewards}')

# Build the model

### Create callback

In [35]:
import os
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker

In [6]:
env_checker.check_env(env)

In [7]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [8]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/' 

In [9]:
callback = TrainAndLoggingCallback(check_freq=1000, save_path=CHECKPOINT_DIR) 

### Build DQN and train

In [10]:
# import DQN algorithm
from stable_baselines3 import DQN

In [15]:
# Create DQN model
model = DQN('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, buffer_size=600000, learning_starts=1000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [19]:
# Training
model.learn(total_timesteps=5000, callback=callback) 

Logging to ./logs/DQN_3
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 25       |
|    ep_rew_mean      | 25       |
|    exploration_rate | 0.81     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2        |
|    time_elapsed     | 42       |
|    total_timesteps  | 100      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 19.2     |
|    ep_rew_mean      | 19.2     |
|    exploration_rate | 0.707    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2        |
|    time_elapsed     | 68       |
|    total_timesteps  | 154      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 19.7     |
|    ep_rew_mean      | 19.7     |
|    exploration_rate | 0.552    |
| time/               |        

FailSafeException: PyDirectInput fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pydirectinput.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

### Test the model

In [38]:
model = DQN.load(os.path.join('train', 'best_model_88000'))

MemoryError: Unable to allocate 9.28 GiB for an array with shape (1200000, 1, 1, 83, 100) and data type uint8

In [22]:
for episode in range(1): 
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(int(action))
        time.sleep(0.01)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))
    time.sleep(2)

Total Reward for episode 0 is 10
